#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [ ]:

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler

import datetime
import pandas as pd
import os
import csv
import operator


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8018), 
                            requestHandler=RequestHandler, allow_none=True)

file_date_pattern = "%Y-%m-%d_%H-%M-%S"

def add_log(string):
    log_file = 'log.csv'
    now = datetime.datetime.now()
    
    f = open(log_file, encoding='utf8')
    length = len(f.readlines())

    f.close()
    if length > 5:
        os.rename(log_file, now.strftime(file_date_pattern) + '.csv')
        pd.DataFrame({
            'Date': [now.strftime('%Y-%m-%d %H:%M:%S')],
            'Type operation': [string]
        }).to_csv(log_file, mode='w', index=False, header=False, encoding='utf8')
    else:
        pd.DataFrame({
            'Date': [now.strftime('%Y-%m-%d %H:%M:%S')],
            'Type operation': [string]
        }).to_csv(log_file, mode='a', index=False, header=False, encoding='utf8')

server.register_function(add_log, 'add_log')


def logs():
    result = list()
    csv_files_names = list(filter(lambda x: x.endswith('.csv'), os.listdir(os.getcwd())))
    csv_files = list(map(lambda x: open(x, encoding='utf8'), csv_files_names))
    for f in csv_files:
        reader = csv.reader(f)
        for line in reader:
            result.append(line)
        f.close()
    
    return result

server.register_function(logs, 'logs')


def logs_slice_by_operation(operation):
    result = list()
    all_logs = logs()
    for log_line in all_logs:
        if operator.eq(log_line[1], operation):
            result.append(log_line)
    return result

server.register_function(logs_slice_by_operation, 'logs_slice_by_operation')

def files_to_logs(csv_files):
    result = list()
    for f in csv_files:
        reader = csv.reader(f)
        for line in reader:
            result.append(line)
        f.close()

    return result


def logs_slice_by_time(left, right, operation):
    date_pattern = '%Y-%m-%d %H:%M:%S'
    left_date = datetime.datetime.strptime(left, date_pattern)
    right_date = datetime.datetime.strptime(right, date_pattern)
    
    csv_files_names = list(filter(lambda x: x.endswith('.csv'), os.listdir(os.getcwd())))
    csv_files_dates = list(map(lambda x: x[0:len(x) - 4], csv_files_names))
    
    csv_files_parsed_dates = list(map(lambda d: datetime.datetime.strptime(d, file_date_pattern), 
                                      list(filter(lambda x: x != 'log', csv_files_dates))))

    csv_files_names.remove('log.csv')
    left_index = 0
    right_index = len(csv_files_names)
    
    for i in range(len(csv_files_parsed_dates) - 1):
        if csv_files_parsed_dates[i] < left_date >= csv_files_parsed_dates[i + 1]:
            left_index = i + 1
        if csv_files_parsed_dates[i] < right_date <= csv_files_parsed_dates[i + 1]:
            right_index = i + 1

    csv_files_names_in_range = csv_files_names[left_index:right_index]
    csv_files = list(map(lambda x: open(x, encoding='utf8'), csv_files_names_in_range))
    logs_from_files = files_to_logs(csv_files)
    if operation is None:
        return logs_from_files
    else:
        result = list()
        
        for log_line in logs_from_files:
            if operator.eq(log_line[1], operation):
                result.append(log_line)
        return result

server.register_function(logs_slice_by_time, 'logs_slice_by_time')



def is_date_in_range(left, right, item):
    return left_date <= item_date and item_date <= right_date

print ("Listening on port 8018...")
server.serve_forever()


Listening on port 8018...


127.0.0.1 - - [04/Dec/2023 21:18:08] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2023 21:18:10] "POST /RPC2 HTTP/1.1" 200 -
